In [89]:
from dotenv import load_dotenv
load_dotenv()
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun
from pydantic import BaseModel, Field
from typing import List



In [90]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)



In [91]:
ddg_tool = DuckDuckGoSearchRun()
ddg_result = ddg_tool.invoke("Tesla competitors and business model")
ddg_result

"Tesla , the insurance business , and non-linear competition . When assessing business competition , it’s critical to look at it, with a long-term eye.Read Also: Tesla Business Model , Elon Musk Companies, Who Owns Tesla , Transitional Business Models , Tesla Competitors . Tesla 's competitors include Ford Motor Company, General Motors Company and more. Discover more competitors and create your own custom side-by-side comparison on Bullfincher. For whatever reason, you may not want a Tesla – for example, if you’re against Musk’s business practices or if you just would prefer an understated electric car – there are now hundreds of great Tesla Model X and 3 competitors . In fact, Tesla 's share of U.S. EV sales dropped below 50% for the first time in 2024, down from a peak of 82.5% in 2019. With competition heating up, investors may want to diversify. Here are seven of the best EV stocks to buy other than Tesla , according to Bank of America What you really want to understand is what is 

In [92]:
from tavily import TavilyClient
import os

tavily_client = TavilyClient(api_key="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")

tavily_result = tavily_client.search(
    query="Tesla latest news financial performance",
    search_depth="advanced"
)

tavily_result


{'query': 'Tesla latest news financial performance',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://finance.yahoo.com/quote/TSLA/',
   'title': 'Tesla, Inc. (TSLA) Stock Price, News, Quote & History - Yahoo Finance',
   'content': '### Performance Overview: TSLA\n\nTrailing total returns as of 1/12/2026, which may include dividends or other distributions. Benchmark is S&P 500 (^GSPC)") .\n\n### YTD Return\n\n### 1-Year Return\n\n### 3-Year Return\n\n### 5-Year Return\n\n### Earnings Trends: TSLA\n\n### Earnings Per Share\n\n### Revenue vs. Earnings\n\nQ4\n\nFY24\n\nQ1\n\nFY25\n\nQ2\n\nFY25\n\nQ3\n\nFY25\n\n### Analyst Insights: TSLA\n\n### Top Analyst\n\n### Analyst Price Targets\n\n### Analyst Recommendations\n\n### Latest Rating\n\n### Statistics: TSLA\n\n### Valuation Measures\n\nMarket Cap\n\n1.49T\n\nEnterprise Value\n\n1.47T\n\nTrailing P/E\n\n309.63\n\nForward P/E\n\n200.00\n\nPEG Ratio (5yr expected)\n\n8.97\n\nPrice/Sales (ttm)\n\n1

In [93]:
research_state = {
    "company": "Tesla",
    "duckduckgo": ddg_result,
    "tavily": tavily_result
}

research_state.keys()


dict_keys(['company', 'duckduckgo', 'tavily'])

In [94]:
class CompanyReport(BaseModel):
    company_overview: str = Field(description="What the company does")
    business_model: str = Field(description="How the company makes money")
    competitors: List[str] = Field(description="Main competitors")
    recent_developments: str = Field(description="Recent news or updates")
    risks: List[str] = Field(description="Business or market risks")
    opportunities: List[str] = Field(description="Growth opportunities")
    stock_context: str = Field(
        description="Stock-related context (descriptive only, no prediction)"
    )


In [95]:
structured_llm = llm.with_structured_output(CompanyReport)


In [96]:
analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a senior market analyst."),
    ("human", """
Using ONLY the research below, generate a structured company report.

DuckDuckGo Research:
{ddg}

Tavily Research:
{tavily}
""")
])



In [97]:
analysis_messages = analysis_prompt.format_messages(
    ddg=research_state["duckduckgo"],
    tavily=research_state["tavily"]
)

company_report = structured_llm.invoke(analysis_messages)
company_report


CompanyReport(company_overview="Tesla, Inc. is an American electric vehicle and clean energy company. The company's main products include electric vehicles, solar panels, and energy storage systems.", business_model="Tesla's business model is based on the design, manufacture, and sale of electric vehicles, solar panels, and energy storage systems. The company also provides services such as vehicle maintenance, charging infrastructure, and energy storage solutions.", competitors=['Ford Motor Company', 'General Motors Company', 'BYD'], recent_developments='Tesla has faced challenges in EV sales and competition, with its share of U.S. EV sales dropping below 50% for the first time in 2024. The company is anticipating the launch of the Cybertruck and has introduced a new, more affordable version of its Model Y SUV.', risks=['Increasing competition in the electric vehicle market', 'Regulatory risks and changes in government policies', 'Supply chain disruptions and material costs'], opportun

In [98]:
company_report.company_overview
company_report.competitors
company_report.risks

['Increasing competition in the electric vehicle market',
 'Regulatory risks and changes in government policies',
 'Supply chain disruptions and material costs']

In [99]:
report_dict = company_report.model_dump()
report_dict


{'company_overview': "Tesla, Inc. is an American electric vehicle and clean energy company. The company's main products include electric vehicles, solar panels, and energy storage systems.",
 'business_model': "Tesla's business model is based on the design, manufacture, and sale of electric vehicles, solar panels, and energy storage systems. The company also provides services such as vehicle maintenance, charging infrastructure, and energy storage solutions.",
 'competitors': ['Ford Motor Company', 'General Motors Company', 'BYD'],
 'recent_developments': 'Tesla has faced challenges in EV sales and competition, with its share of U.S. EV sales dropping below 50% for the first time in 2024. The company is anticipating the launch of the Cybertruck and has introduced a new, more affordable version of its Model Y SUV.',
 'risks': ['Increasing competition in the electric vehicle market',
  'Regulatory risks and changes in government policies',
  'Supply chain disruptions and material costs']

In [100]:
import sqlite3
import json
from datetime import datetime
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
conn = sqlite3.connect("company_reports.db")
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS reports (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    company TEXT,
    report_json TEXT,
    created_at TEXT
)
""")
conn.commit()


In [101]:
cursor.execute("""
INSERT INTO reports (company, report_json, created_at)
VALUES (?, ?, ?)
""", (
    research_state["company"],
    json.dumps(report_dict),
    datetime.utcnow().isoformat()
))

conn.commit()


C:\Users\TEJAS\AppData\Local\Temp\ipykernel_2628\3570507597.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime.utcnow().isoformat()


In [102]:
cursor.execute("""
SELECT company, report_json, created_at
FROM reports
ORDER BY created_at DESC
LIMIT 1
""")

row = cursor.fetchone()
row


('Tesla',
 '{"company_overview": "Tesla, Inc. is an American electric vehicle and clean energy company. The company\'s main products include electric vehicles, solar panels, and energy storage systems.", "business_model": "Tesla\'s business model is based on the design, manufacture, and sale of electric vehicles, solar panels, and energy storage systems. The company also provides services such as vehicle maintenance, charging infrastructure, and energy storage solutions.", "competitors": ["Ford Motor Company", "General Motors Company", "BYD"], "recent_developments": "Tesla has faced challenges in EV sales and competition, with its share of U.S. EV sales dropping below 50% for the first time in 2024. The company is anticipating the launch of the Cybertruck and has introduced a new, more affordable version of its Model Y SUV.", "risks": ["Increasing competition in the electric vehicle market", "Regulatory risks and changes in government policies", "Supply chain disruptions and material c

In [104]:
company_name = row[0]
report_data = json.loads(row[1])
created_at = row[2]

company_name, created_at



('Tesla', '2026-01-13T09:31:13.982632')

In [105]:
file_name = f"{company_name}_Market_Report.pdf"

pdf = SimpleDocTemplate(
    file_name,
    pagesize=A4
)

styles = getSampleStyleSheet()
content = []
content.append(
    Paragraph(
        f"<b>{company_name} – Market Research Report</b>",
        styles["Title"]
    )
)

content.append(Spacer(1, 20))

content.append(
    Paragraph(
        f"Generated on: {created_at}",
        styles["Normal"]
    )
)

content.append(Spacer(1, 20))


In [106]:
def add_section(title, text):
    content.append(Paragraph(f"<b>{title}</b>", styles["Heading2"]))
    content.append(Spacer(1, 10))
    content.append(Paragraph(text, styles["Normal"]))
    content.append(Spacer(1, 15))

add_section("Company Overview", report_data["company_overview"])
add_section("Business Model", report_data["business_model"])

add_section(
    "Competitors",
    ", ".join(report_data["competitors"])
)

add_section("Recent Developments", report_data["recent_developments"])

add_section(
    "Risks",
    ", ".join(report_data["risks"]) if report_data["risks"] else "Not available"
)

add_section(
    "Opportunities",
    ", ".join(report_data["opportunities"]) if report_data["opportunities"] else "Not available"
)

add_section("Stock Context", report_data["stock_context"])

pdf.build(content)

file_name


'Tesla_Market_Report.pdf'

In [107]:
import os

os.path.exists(file_name)


True